In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# !pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Spotify API
CLIENT_ID = "1656aba538ac46ec90faeee4214f0169"
CLIENT_SECRET = "7cf73c012bef4557a88b8d98717cabda"

auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

def get_song_features(song_name, artist_name):
    try:
        # Search for songs
        query = f"track:{song_name} artist:{artist_name}"
        result = sp.search(q=query, type="track", limit=1)

        # examine the result
        if not result['tracks']['items']:
            return f"Did not find songs：{song_name} by {artist_name}"

        # get track id
        track_info = result['tracks']['items'][0]
        track_id = track_info['id']
        popularity = track_info['popularity']
        print(f"Find Songs：{result['tracks']['items'][0]['name']} by {result['tracks']['items'][0]['artists'][0]['name']}")

        # get features
        features = sp.audio_features([track_id])[0]
        if features:
            features['popularity'] = popularity
        return features
    except Exception as e:
        return f"Error：{e}"


In [ ]:
import requests
from bs4 import BeautifulSoup

def get_top_songs(year):

    url = "https://kworb.net/spotify/songs_" + f"{year}.html"
    response = requests.get(url)

    if response.status_code != 200:
        print("Failed to retrieve data")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    song_list = []

    table = soup.find('table')
    rows = table.find('tbody').find_all('tr')

    for row in rows:
        columns = row.find_all('td')
        #print(columns)

        artist_title = columns[0].text.strip()
        streams = columns[1].text.strip()
        daily = columns[2].text.strip()

        song_list.append((artist_title, streams, daily))

    spotify_data = pd.DataFrame(song_list, columns=['Artist and Title', 'Streams', 'Daily Streams'])

    return spotify_data

In [ ]:
def extract_features(row):
    features = row['feature']
    if isinstance(features, dict):
        # divide features into different columns
        for key, value in features.items():
            row[key] = value
    return row

In [ ]:
# replaced 2021 with years 2018-2023
# to generate all of our data files
spotify_2021 = get_top_songs(2021)
spotify_2021

spotify_2021['Artist and Title'] = spotify_2021['Artist and Title'].str.split(' - ')
spotify_2021['artist_name'] = spotify_2021['Artist and Title'].apply(lambda x: x[0])
spotify_2021['song_name'] = spotify_2021['Artist and Title'].apply(lambda x: x[1])

spotify_2021['artist_name'] = spotify_2021['artist_name'].apply(lambda x: x.lower())
spotify_2021['song_name'] = spotify_2021['song_name'].apply(lambda x: x.lower())
spotify_2021

,Artist and Title,Streams,Daily Streams,artist_name,song_name
0,"[The Kid LAROI, STAY]","3,347,184,330","1,473,805",the kid laroi,stay
1,"[Olivia Rodrigo, drivers license]","2,325,657,574","929,699",olivia rodrigo,drivers license
2,"[Olivia Rodrigo, good 4 u]","2,298,333,269","835,756",olivia rodrigo,good 4 u
3,"[Maroon 5, Memories]","2,126,410,268","954,444",maroon 5,memories
4,"[Lil Nas X, INDUSTRY BABY]","2,088,217,488","471,972",lil nas x,industry baby
...,...,...,...,...,...
995,"[OneRepublic, Wanted]","108,557,612","11,980",onerepublic,wanted
996,"[Lovejoy, Sex Sells]","108,539,295","13,061",lovejoy,sex sells
997,"[Electric Callboy, We Got the Moves]","108,394,895","89,951",electric callboy,we got the moves
998,"[Morat, No Hay Más Que Hablar]","108,239,542","100,634",morat,no hay más que hablar


In [ ]:
# get features for each song
import time
batch_size = 10
for i in range(0, len(spotify_2021), batch_size):
    batch = spotify_2021.iloc[i:i+batch_size]
    batch['feature'] = batch[['song_name', 'artist_name']].apply(
        lambda x: get_song_features(x['song_name'], x['artist_name']), axis=1
    )
    # 将结果更新到主数据框
    spotify_2021.loc[batch.index, 'feature'] = batch['feature']
    time.sleep(1)  # 增加间隔以避免速率限制

# divided features into different columns
spotify_2021 = spotify_2021.apply(extract_features, axis=1)
spotify_2021.drop(columns=['feature'], inplace=True)

# data
spotify_2021

Find Songs：STAY (with Justin Bieber) by The Kid LAROI
Find Songs：drivers license by Olivia Rodrigo
Find Songs：good 4 u by Olivia Rodrigo
Find Songs：Memories by Maroon 5
Find Songs：INDUSTRY BABY (feat. Jack Harlow) by Lil Nas X
Find Songs：MONTERO (Call Me By Your Name) by Lil Nas X
Find Songs：Mood (feat. iann dior) by 24kGoldn
Find Songs：Kiss Me More (feat. SZA) by Doja Cat
Find Songs：Easy On Me by Adele
Find Songs：Bad Habits by Ed Sheeran


/var/folders/xf/t9mdp7cn0f911_dgjxqwf0n80000gn/T/ipykernel_1981/2435410206.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['feature'] = batch[['song_name', 'artist_name']].apply(


Find Songs：Ghost by Justin Bieber
Find Songs：deja vu by Olivia Rodrigo
Find Songs：Woman by Doja Cat
Find Songs：traitor by Olivia Rodrigo
Find Songs：Shivers by Ed Sheeran
Find Songs：Peaches (feat. Daniel Caesar & Giveon) by Justin Bieber
Find Songs：Enemy (with JID) - from the series Arcane League of Legends by Imagine Dragons
Find Songs：Pepas by Farruko
Find Songs：Todo De Ti by Rauw Alejandro
Find Songs：Yonaguni by Bad Bunny


/var/folders/xf/t9mdp7cn0f911_dgjxqwf0n80000gn/T/ipykernel_1981/2435410206.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['feature'] = batch[['song_name', 'artist_name']].apply(


Find Songs：Happier Than Ever by Billie Eilish
Find Songs：Leave The Door Open by Bruno Mars
Find Songs：Where Are You Now by Lost Frequencies
Find Songs：My Universe by Coldplay
Find Songs：Butter by BTS
Find Songs：happier by Olivia Rodrigo
Find Songs：MONEY by LISA
Find Songs：Freaks by Surf Curse
Find Songs：Need to Know by Doja Cat
Find Songs：favorite crime by Olivia Rodrigo


...

Max Retries reached


Find Songs：Slumber Party (feat. Princess Nokia) by Ashnikko


Max Retries reached


Find Songs：WUSYANAME (feat. Youngboy Never Broke Again & Ty Dolla $ign) by Tyler, The Creator


KeyboardInterrupt: 

In [ ]:
print(len(spotify_2021))
spotify_2021.info()
spotify_2021.to_csv('spotify_2021.csv')

941
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Artist and Title  941 non-null    object 
 1   Daily Streams     941 non-null    object 
 2   Streams           941 non-null    object 
 3   acousticness      939 non-null    float64
 4   analysis_url      939 non-null    object 
 5   artist_name       941 non-null    object 
 6   danceability      939 non-null    float64
 7   duration_ms       939 non-null    float64
 8   energy            939 non-null    float64
 9   id                939 non-null    object 
 10  instrumentalness  939 non-null    float64
 11  key               939 non-null    float64
 12  liveness          939 non-null    float64
 13  loudness          939 non-null    float64
 14  mode              939 non-null    float64
 15  popularity        939 non-null    float64
 16  song_name         941 non-null    object